In [127]:
import nltk
from collections import defaultdict, Counter
import gensim, logging
import nltk
from gensim.models.keyedvectors import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
from multiprocessing import Process, Pool
import os
import collections
from random import shuffle
from copy import deepcopy
from collections import OrderedDict
import pickle
import networkx as nx
import matplotlib.pyplot as plt

In [128]:
# nltk.corpus.gutenberg.fileids()
# total_corpus = []
# for file in nltk.corpus.gutenberg.fileids():
#     with open((nltk.corpus.gutenberg.words(file).fileid.path), 'r') as f:
#         for line in f.readlines():
#             try:
#                 total_corpus.append(line.split())
#             except:
#                 print ("hi")
#                 print (line)

# with open(emma.fileid.path) as f:
#     for line in f.readlines():
#         print (line)

# emma = nltk.corpus.gutenberg.words('austen-emma.txt')
# vocab = list(set(emma))
# len(vocab)

In [129]:
def extract_patterns_in_words(patterns,pattern_counter,word1,word2,max_len):
    i = 1
    while(word1[:i] == word2[:i]):
        i = i + 1
    if i != 1 and i > max(len(word1[i-1:]), len(word2[i-1:])) < max_len:
        pattern_counter[("suffix",word1[i-1:], word2[i-1:])] += 1
        if ("suffix",word1[i-1:], word2[i-1:]) in patterns:
            patterns[("suffix",word1[i-1:], word2[i-1:])].append((word1, word2))
        else:
            patterns[("suffix",word1[i-1:], word2[i-1:])] = [(word1, word2)]
#         patterns[("suffix",word1[i-1:], word2[i-1:], word1, word2)] += 1
    i = 1
    while(word1[-i:] == word2[-i:]):
        i = i + 1
    if i != 1 and max(len(word1[:-i+1]), len(word2[:-i+1])) < max_len:
        pattern_counter[("prefix",word1[:-i+1], word2[:-i+1])] += 1
        if ("prefix",word1[:-i+1], word2[:-i+1]) in patterns:
            patterns[("prefix",word1[:-i+1], word2[:-i+1])].append((word1, word2))
        else:
            patterns[("prefix",word1[:-i+1], word2[:-i+1])] = [(word1, word2)]
#         patterns[("prefix",word1[:-i+1], word2[:-i+1], word1, word2)] += 1
    return patterns

In [130]:
def build_pattern_dict(vocab,max_len = 6):
    patterns  = defaultdict(list)
#     print (patterns)
    pattern_counter = Counter()
    for word in vocab:
        for second_word in vocab:
            if word != second_word:
                extract_patterns_in_words(patterns,pattern_counter,word,second_word,max_len)
    return patterns, pattern_counter

In [131]:
def downsample_patterns():
    #Downsample to include only top 1000
    pattern_1000 = defaultdict(list)
    for pattern,items in patterns.items():
        shuffle(items)
        pattern_1000[pattern] = items[:1000]
    return pattern_1000

In [132]:
def pair_wise_similarity(word_pair1, word_pair2,annoy_index=None, topn = 10,):
    closest_n = word_vectors.most_similar(positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]], topn=topn)
#     print (word_pair2[1])
#     print (closest_n)
    for word, cos_sim in closest_n:
        if word == word_pair2[1]:
            return True
    return False

def annoy_pair_wise_similarity(word_pair1, word_pair2,annoy_index, topn = 10):
    closest_n = word_vectors.most_similar(positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]], topn=topn, indexer=annoy_index)
#     print (word_pair2[1])
#     print (closest_n)
    for word, cos_sim in closest_n:
        if word == word_pair2[1]:
            return True
    return False

def get_similarity_rank(word_pair1, word_pair2, topn=500):
    closest_n = word_vectors.most_similar(positive=[word_pair2[0], word_pair1[1]], negative=[word_pair1[0]], topn=topn)
#     print (word_pair2[1])
#     print (closest_n)
    for n,(word, cos_sim) in enumerate(closest_n):
        if word == word_pair2[1]:
            return (n, cos_sim)
    return (None, None)

In [133]:
def index_vector(word_vectors, dimensions=300):
    fname = '../data/annoy.index'
    # Persist index to disk
    if os.path.exists(fname):
        annoy_index = AnnoyIndexer()
        annoy_index.load(fname)
        annoy_index.model = word_vectors
    else:
        annoy_index = AnnoyIndexer(word_vectors, dimensions)
        annoy_index.save(fname)
    return annoy_index

In [150]:
def get_hit_rate(patterns, similarity_function):
    if os.path.exists('../data/hitrate_'+ str(len(word_vectors.vocab))):
        hit_rate_file_r = open('../data/hitrate_'+ str(len(word_vectors.vocab)), 'rb')
        hit_rates_rules = pickle.load(hit_rate_file_r)
        return hit_rates_rules
    else:
        hit_rate_file_w = open('../data/hitrate_'+ str(len(word_vectors.vocab)),"wb" )
        hit_rates_rules = {}
        for (pattern,support_set) in patterns.items():
            hit_rates_word_pair = {}
            for pair1 in support_set:
                hit_count = 0
                hit_pairs = set()
                for pair2 in support_set:
                    if pair1 != pair2 and similarity_function(pair1, pair2, 10):
                        hit_count += 1
                        hit_pairs.add(pair2)
                if len(support_set) ==1:
                    total = 1
                else:
                    total = len(support_set) - 1
                if hit_count != 0:
                    hit_rates_word_pair[pair1] =  hit_pairs
            if len(support_set) != 1 and hit_rates_word_pair:
                hit_rates_rules[pattern] = hit_rates_word_pair
        pickle.dump(hit_rates_rules, hit_rate_file_w)
        hit_rate_file_w.close()
        return hit_rates_rules

In [151]:
#pattern_counter.most_common()[:-20:-1]

In [166]:
%time word_vectors = KeyedVectors.load_word2vec_format('/home/raja/models/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=5000)

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 108 ms


In [167]:
%time patterns, pattern_counter = build_pattern_dict(word_vectors.vocab.keys())

CPU times: user 14 s, sys: 12 ms, total: 14 s
Wall time: 14 s


In [168]:
pattern_counter.most_common(20)

[(('suffix', 's', ''), 598),
 (('suffix', '', 's'), 598),
 (('suffix', '', 'ed'), 168),
 (('suffix', 'ed', ''), 168),
 (('suffix', '', 'ing'), 149),
 (('suffix', 'ing', ''), 149),
 (('suffix', 'ing', 'ed'), 141),
 (('suffix', 'ed', 'ing'), 141),
 (('suffix', '', 'd'), 127),
 (('suffix', 'd', ''), 127),
 (('suffix', 's', 'ed'), 86),
 (('suffix', 'ed', 's'), 86),
 (('suffix', '', 'ly'), 80),
 (('suffix', 'ly', ''), 80),
 (('suffix', 's', 'ing'), 76),
 (('suffix', 'ing', 's'), 76),
 (('suffix', 'e', 'ing'), 74),
 (('suffix', 'ing', 'e'), 74),
 (('suffix', 's', 'd'), 62),
 (('suffix', 'd', 's'), 62)]

In [169]:
%time sampled_patterns = downsample_patterns()

CPU times: user 2.07 s, sys: 24 ms, total: 2.1 s
Wall time: 2.1 s


In [170]:
# word_vectors_annoy = deepcopy(word_vectors)
# word_vectors.init_sims(replace=True)
# %time annoy_index = index_vector(word_vectors=word_vectors, dimensions=300)
# %time hit_rates = calculate_hit_rate(sampled_patterns, annoy_pair_wise_similarity, annoy_index)

In [171]:
#pair_wise_similarity(("asking", "banking"), ("ask", "bank"))

In [172]:
%time hit_rates = get_hit_rate(sampled_patterns, pair_wise_similarity)

CPU times: user 14min 58s, sys: 2.18 s, total: 15min
Wall time: 7min 30s


In [173]:
# def temp(pattern, transformations):
# #     ll = [transformations[transformation] for transformation in transformations]
# #     print (ll)
# #     print (len(ll), len(transformations))
#     print (transformations)
#     prototypes = []
#     support_set = set(sampled_patterns[pattern])
# #     print (pattern)
# #     print (support_set)
#     while True:
#         explains_by_count = sorted(transformations.items(), key=lambda kv: -len(kv[1]))
#         print (explains_by_count)
#         best = explains_by_count[0]
#         if len(best[1]) >= 10: #The prototype explains more than 10 word pairs
#             prototypes.append((best[0][0], len(best[1]) / float(len(support_set))))
#         else:
#             break
#         del transformations[best[0]]
#         #Remove all explained pairs from the support set
#         support_set = support_set - best[1]
#         for k, v in transformations.items():
#             transformations[k] = transformations[k] - best[1]
#         explains_by_count.pop(0)

#         if not (len(support_set) >= 10 and len(explains_by_count) and explains_by_count[0] >= 10):
#             break
#     print (prototypes)

In [174]:
#hit_rates

In [175]:
cons = 4
def update_morpho_rules(patterns):
    global morphological_rules
    for pattern in patterns:
        transformations = patterns[pattern]
#         ll = [transformations[transformation] for transformation in transformations]
    #     print (ll)
    #     print (len(ll), len(transformations))
        support_set = set(sampled_patterns[pattern])
        while(True):
            transformations_by_count = sorted(transformations.items(), key=lambda kv: len(kv[1]), reverse=True)
            best = transformations_by_count[0]
    #         print (transformations_by_count)
    #         print (transformations)
            if len(best[1]) >= cons:
                morphological_rules[best[0]] = (pattern, len(best[1]) / float(len(support_set)),  best[1])
    #             directions.append(best)
                del transformations[best[0]]
            else:
                break

            #Remove all explained pairs from the support set
            #TODO: Remove best[0] from support set and transformations
            support_set = support_set - best[1]
            for k, v in transformations.items():
    #             print ("*"*50)
    #             print (transformations[k])
                transformations[k] = transformations[k] - best[1]
    #             print (transformations[k])
    #             print ("__"*50)

            transformations_by_count.pop(0)
            if not (len(support_set) >= cons and len(transformations_by_count) and len(transformations_by_count[0][1]) >= cons):
                break
    #     return directions
    #         print (best_transformation,best_support)


In [176]:
#%time patterns = calculate_hit_rate(sampled_patterns, pair_wise_similarity)
morphological_rules = {}
%time update_morpho_rules(hit_rates)

CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 130 ms


In [177]:
print (len(morphological_rules))

234


In [178]:
morphological_rules

{('##', '###'): (('prefix', '', '#'),
  0.6875,
  {('####', '#####'),
   ('##,###', '###,###'),
   ('##.#', '###.#'),
   ('##.##', '###.##'),
   ('##th', '###th'),
   ('#,###', '##,###'),
   ('#-#', '##-#'),
   ('#-##', '##-##'),
   ('#.#', '##.#'),
   ('#.##', '##.##'),
   ('#:##', '##:##')}),
 ('###', '##'): (('prefix', '#', ''),
  0.6875,
  {('#####', '####'),
   ('###,###', '##,###'),
   ('###.#', '##.#'),
   ('###.##', '##.##'),
   ('###th', '##th'),
   ('##,###', '#,###'),
   ('##-#', '#-#'),
   ('##-##', '#-##'),
   ('##.#', '#.#'),
   ('##.##', '#.##'),
   ('##:##', '#:##')}),
 ('####', '##'): (('suffix', '##', ''),
  0.8,
  {('#####', '###'),
   ('####-####', '####-##'),
   ('#.###', '#.#'),
   ('#.####', '#.##')}),
 ('####', '###'): (('suffix', '#', ''),
  0.75,
  {('###', '##'),
   ('###-####', '###-###'),
   ('###.##', '###.#'),
   ('##-##', '##-#'),
   ('##.##', '##.#'),
   ('#-##', '#-#'),
   ('#.##', '#.#'),
   ('#.###', '#.##'),
   ('#.####', '#.###')}),
 ('####-##', '#

In [179]:
for item in morphological_rules:
    print (item, morphological_rules[item])
    print ("*" * 50)

('in', 'is') (('suffix', 'n', 's'), 0.5454545454545454, {('taken', 'takes'), ('shown', 'shows'), ('known', 'knows'), ('given', 'gives'), ('seen', 'sees'), ('In', 'Is')})
**************************************************
('investors', 'Investors') (('prefix', 'i', 'I'), 0.9411764705882353, {('index', 'Index'), ('international', 'International'), ('industry', 'Industry'), ('insurance', 'Insurance'), ('it', 'It'), ('information', 'Information'), ('instead', 'Instead'), ('investigators', 'Investigators'), ('internet', 'Internet'), ('if', 'If'), ('its', 'Its'), ('investment', 'Investment'), ('is', 'Is'), ('indeed', 'Indeed'), ('in', 'In'), ('island', 'Island')})
**************************************************
('R', 'R.') (('suffix', '', '.'), 0.7333333333333333, {('J', 'J.'), ('Dr', 'Dr.'), ('E', 'E.'), ('S', 'S.'), ('Ms', 'Ms.'), ('C', 'C.'), ('Mr', 'Mr.'), ('M', 'M.'), ('D', 'D.'), ('St', 'St.'), ('A', 'A.')})
**************************************************
('given', 'give') (('suf

In [180]:
for rule in morphological_rules:
    print (rule)
    break

('in', 'is')


In [181]:
# if os.path.exists('../data/rules_'+ str(len(word_vectors.vocab))):
#     rule_file_r = open('../data/rules_'+ str(len(word_vectors.vocab)), 'rb')
#     morphological_rules = pickle.load(rule_file_r)
# else:
#     hit_rates = calculate_hit_rate(sampled_patterns, pair_wise_similarity)
#     hit_rate_file_w = open('../data/rules_'+ str(len(word_vectors.vocab)),"wb" )
#     pickle.dump(hit_rates, hit_rate_file_w)
#     hit_rate_file_w.close()
# # %time hit_rates = calculate_hit_rate(sampled_patterns, pair_wise_similarity)

In [182]:
G = nx.MultiDiGraph()

In [183]:
%time G.add_nodes_from(word_vectors.vocab.keys())

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.32 ms


In [184]:
for dw,support in morphological_rules.items():
    morp_rule, hit_rate,support_set = support
    (word1, word2) = dw
    for (word3, word4) in support_set:
        (rank,cos_sim) = get_similarity_rank((word1,word2),(word3,word4))
        if rank < 3 and cos_sim > 0.5:
            if not G.has_edge(word3,word4,key=dw):
                G.add_edge(word3,word4,key=dw,cos=cos_sim,rank=rank)
        else:
#             print (rank,cos_sim, word3, word2, dw)
            pass

#         G.add_edge(word1, word2 )

In [185]:
for item in G['make']:
    print (item)
    d = G['make'][item]
    print(d)
print (G.out_edges('make',data=True))
print (G.out_edges('make',keys=True))

makes
{('highlight', 'highlights'): {'cos': 0.664941668510437, 'rank': 0}}
making
{('include', 'including'): {'cos': 0.6760380268096924, 'rank': 0}}
drive
{('making', 'driving'): {'cos': 0.5898005366325378, 'rank': 0}}
[('make', 'makes', {'cos': 0.664941668510437, 'rank': 0}), ('make', 'making', {'cos': 0.6760380268096924, 'rank': 0}), ('make', 'drive', {'cos': 0.5898005366325378, 'rank': 0})]
[('make', 'makes', ('highlight', 'highlights')), ('make', 'making', ('include', 'including')), ('make', 'drive', ('making', 'driving'))]


In [186]:
print (len(G.edges))
print (len(G.nodes))

2770
5000


In [187]:
for item in G.nodes:
    if G.out_degree(item) > 1:
        print (item, G.out_degree(item))

## 2
get 2
game 2
#.# 2
any 3
make 3
state 2
government 2
your 2
work 3
take 4
including 2
#,### 2
#.## 3
play 3
help 3
##,### 2
know 2
end 3
want 3
school 2
need 2
win 2
use 5
##.# 2
run 2
come 3
city 2
###,### 2
You 2
support 2
program 2
family 2
##-## 2
show 3
services 2
every 2
service 2
start 3
##.## 2
industry 2
call 3
students 2
making 3
plan 2
look 3
give 3
open 2
record 2
reported 2
court 2
working 3
keep 2
find 2
increase 2
looking 2
provide 3
include 2
continue 3
played 4
move 3
become 3
loss 2
coach 3
change 3
taken 2
asked 4
started 2
director 2
party 2
taking 3
project 2
scored 2
coming 2
building 2
following 3
return 4
face 3
received 2
rate 2
offer 3
playing 5
using 2
needs 2
fire 2
feel 2
result 2
reports 4
investment 3
comes 2
runs 2
visit 3
bring 2
makes 3
vote 3
town 2
increased 2
charges 2
hope 3
attack 3
region 2
worked 3
fall 2
changes 2
includes 3
allowed 2
An 2
education 2
hold 2
turn 4
finished 2
wants 3
turned 2
bank 2
district 2
allow 3
helped 2
officer 2
sh

In [188]:
for node in list(G.nodes):
    for neighbor in list(G.neighbors(node)):
        if word_vectors.vocab[node].count > word_vectors.vocab[neighbor].count:
            if G.has_edge(neighbor, node):
                G.remove_edge(neighbor, node)
        else:
            if G.has_edge(node, neighbor):
                G.remove_edge(node, neighbor)
                print (G.number_of_edges(node,neighbor))
        if G.number_of_edges(node,neighbor) > 1:
            print ("AHHH", neighbor, node)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [122]:
G.get_edge_data('reported', 'started')

{('report', 'start'): {'cos': 0.5782519578933716, 'rank': 0}}

In [123]:
G.get_edge_data('started','reported')

In [124]:
print (len(G.nodes))
print (len(G.edges))

5000
49


In [125]:
for item in G.edges():
    print (item)

('as', 'an')
('some', 'someone')
('make', 'drive')
('make', 'write')
('where', 'somewhere')
('use', 'vote')
('use', 'drive')
('set', 'put')
('come', 'become')
('used', 'voted')
('used', 'served')
('came', 'became')
('every', 'everyone')
('until', 'Until')
('making', 'writing')
('reported', 'started')
('working', 'playing')
('played', 'started')
('coming', 'becoming')
('rate', 'save')
('using', 'serving')
('using', 'writing')
('player', 'worker')
('sent', 'send')
('spent', 'spend')
('create', 'operate')
('operating', 'creating')
('success', 'successful')
('talk', 'walk')
('everyone', 'anyone')
('users', 'voters')
('created', 'operated')
('driving', 'writing')
('talking', 'walking')
('walked', 'talked')
('user', 'maker')
('defeat', 'defeated')
('uses', 'serves')
('uses', 'writes')
('discovered', 'discovery')
('reporting', 'supporting')
('Every', 'Everyone')
('rating', 'saving')
('sets', 'puts')
('Any', 'Anyone')
('operates', 'creates')
('delayed', 'delays')
('covering', 'covers')
('anywh

In [38]:
M = nx.MultiGraph()
M.add_edge(1,2,weight=19)
M.add_edge(1,2,weight=7)
M.add_edge(2,3,weight=42)

G2 = nx.Graph(M,)

In [39]:
G2.edges(data=True)

EdgeDataView([(1, 2, {'weight': 7}), (2, 3, {'weight': 42})])